In [ ]:
import torchvision , torch
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor
from rcnn_transfrom import InterpolationTransform as it

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device = 'cuda:0'
num_classes = 2

def get_instance_segmentation_model(num_classes,pretrained=True,mode='bilinear'):
    # load an instance segmentation model pre-trained on COCO
    model = torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained)

    # get the number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    # now get the number of input features for the mask classifier
    in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
    hidden_layer = 256
    # and replace the mask predictor with a new one
    model.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask,
                                                       hidden_layer,
                                                       num_classes)
    model.transform = it(min_size=(800,), max_size=1333,image_mean=[0.485, 0.456, 0.406],image_std=[0.229, 0.224, 0.225],mode=mode)
    return model

In [ ]:
bicubic_m = get_instance_segmentation_model(num_classes,pretrained=False,mode='bicubic')
bicubic_m.to(device)
m_path = './model/pf_4_bicubic.pth'
bicubic_pter = torch.load(m_path)
# bicubic_m.eval()
# bicubic_m.load_state_dict(bicubic_pter['model_state_dict'])

nearest_m = get_instance_segmentation_model(num_classes,pretrained=False,mode='nearest')
nearest_m.to(device)
m_path = './model/pf_4_nearest.pth'
nearest_pter = torch.load(m_path)
# nearest_m.eval()
# nearest_m.load_state_dict(nearest_pter['model_state_dict'])

device = 'cuda:1'
bilinear_m    = get_instance_segmentation_model(num_classes,pretrained=False,mode='bilinear')
bilinear_m.to(device)
m_path        = './model/pf_4_bilinear.pth'
bilinear_pter = torch.load(m_path)
# bilinear_m.eval()
# bilinear_m.load_state_dict(bilinear_pter['model_state_dict'])

bicubic_evals  = bicubic_pter['evaluators']
nearest_evals  = nearest_pter['evaluator']
bilinear_evals = bilinear_pter['evaluators']

In [ ]:
from matplotlib import pyplot as plt
interpols = ['bicubic','nearest','bilinear']
metric_name   = ['ap','ap50','ap75','Maskap','Maskap50','Maskap75']
num_metric    = len(metric_name)
limit         = 10
line_styles   = ['-','-.',':','--']
legend_sizes  = ['xx-small', 'x-small', 'small', 'medium', 'large', 'x-large', 'xx-large']

f, axs = plt.subplots(num_metric, 1, figsize=(9, 15),sharex=False) # w,h

for j,inter in enumerate(interpols): # ['bicubic','nearest','bilinear']
    exec(f'evaluators={inter}_evals')
    # metrics = {'ap':[], 'ap50':[], 'ap75':[], ..., 'Maskap75':[]}
    metrics = dict(zip( metric_name, list( [] for _ in range(len(metric_name))  )  
                      ))
    # to fill metrics
    for evtor in evaluators:
        for iou_type, coco_eval in evtor.coco_eval.items():
            #coco_eval.summarize()
            #  iou_type : 'bbox' ,'segm'
            if iou_type == 'bbox':
                aps = coco_eval.stats[:3]
                metrics['ap'].append(aps[0])
                metrics['ap50'].append(aps[1])
                metrics['ap75'].append(aps[2])

            else : # 'segm'
                aps = coco_eval.stats[:3]
                metrics['Maskap'].append(aps[0])
                metrics['Maskap50'].append(aps[1])
                metrics['Maskap75'].append(aps[2])
    
    metrics_keys = list(metrics.keys() )
    for i,met_name in enumerate(metrics):
        axs[i].set_title(met_name,fontdict={'fontsize':20})
        axs[i].plot( metrics[met_name][:limit],linestyle=line_styles[j],label=inter)
        axs[i].legend(loc='lower right',fontsize=legend_sizes[4] )
plt.subplots_adjust(left=0.125, bottom=0.1,  right=0.9, top=0.9, wspace=0.2, hspace=1)
# plt.subplots(constrained_layout=True)
    #         coco_eval.stats[:3] # ap ,ap0.5 , ap0.75

# plt.savefig('assets/pf3.jpg')

In [ ]:
import pandas as pd
num_epoch = 11
aps = []
for inter in interpols:
    exec(f"bbox = {inter}_evals[num_epoch].coco_eval['bbox'].stats[:3]")
    exec(f"segm = {inter}_evals[num_epoch].coco_eval['segm'].stats[:3]")
    row = bbox.tolist()+segm.tolist()
    aps.append(row)
pd.DataFrame(aps,index=interpols,columns=metric_name)